In [41]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [42]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else 'cpu'
)

print(device)

mps


In [43]:
# FC and Relu
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # To make 1 dimension
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU(),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [44]:
model = NeuralNetwork().to(device='mps')
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [45]:
# softmax(classify)

x = torch.rand(1, 28, 28, device=device)
logits = model(x)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)

print(f'Predicted class: {y_pred}')


Predicted class: tensor([2], device='mps:0')


In [46]:
# layer

input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [47]:
# flatten

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [48]:
# linear

layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [49]:
# ReLu

print(f'Before ReLU: {hidden1}\n\n')

hidden1 = nn.ReLU()(hidden1)

print(f'After ReLU: {hidden1}')

Before ReLU: tensor([[-1.0506e-01, -3.0989e-01,  1.8726e-01, -1.4629e-01, -4.1669e-01,
          3.4275e-01, -2.8735e-02, -6.9294e-01, -5.4847e-02,  1.4303e-01,
          2.2102e-01, -4.3851e-01,  1.7006e-01,  1.7458e-01, -2.4902e-01,
          2.8003e-02,  4.7930e-01, -3.0579e-01,  4.0274e-02,  4.1201e-01],
        [-2.0256e-01, -4.0367e-01,  3.1029e-01, -1.0780e-01, -2.8202e-01,
          7.0549e-01, -4.4443e-01, -7.0698e-01, -2.6976e-01, -3.4651e-01,
          4.2472e-01, -4.0351e-01,  1.6199e-01,  1.3469e-01, -4.9725e-01,
         -3.4175e-02, -6.9464e-04, -1.2084e-01, -2.4637e-01,  6.9760e-01],
        [-1.5589e-01, -8.5568e-02,  4.8741e-01, -1.3863e-01, -3.1065e-01,
          1.8929e-01, -4.4343e-01, -7.2198e-01, -8.0412e-02,  7.5898e-02,
          3.3217e-01, -5.8602e-01,  4.6109e-01,  6.9992e-02, -2.1211e-01,
         -6.5773e-03,  9.3771e-02, -2.0321e-01, -2.9575e-01,  4.6538e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.1873, 0.0000, 0.0000,

In [50]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10),
    nn.ReLU(),
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
print(logits)

tensor([[0.2447, 0.2795, 0.0176, 0.0000, 0.2124, 0.3247, 0.0144, 0.0218, 0.0000,
         0.0645],
        [0.2136, 0.1499, 0.0272, 0.0000, 0.1726, 0.2595, 0.0000, 0.0879, 0.0000,
         0.0715],
        [0.2269, 0.3389, 0.0000, 0.0000, 0.2954, 0.2176, 0.0554, 0.0696, 0.0000,
         0.0409]], grad_fn=<ReluBackward0>)


In [51]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

print(pred_probab)

tensor([[0.1126, 0.1166, 0.0897, 0.0882, 0.1090, 0.1220, 0.0895, 0.0901, 0.0882,
         0.0941],
        [0.1118, 0.1049, 0.0928, 0.0903, 0.1073, 0.1170, 0.0903, 0.0986, 0.0903,
         0.0970],
        [0.1099, 0.1229, 0.0876, 0.0876, 0.1177, 0.1089, 0.0926, 0.0939, 0.0876,
         0.0913]], grad_fn=<SoftmaxBackward0>)


In [52]:
print(f'Model structure: {model}\n\n')

for name, param in model.named_parameters():
    print(f'Layer: {name} : Size: {param.size()} : values : {param[:2]} \n')

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


Layer: linear_relu_stack.0.weight : Size: torch.Size([512, 784]) : values : tensor([[-0.0032,  0.0116,  0.0101,  ..., -0.0132,  0.0356,  0.0185],
        [-0.0210, -0.0303,  0.0280,  ..., -0.0039, -0.0113,  0.0004]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias : Size: torch.Size([512]) : values : tensor([-0.0116,  0.0282], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight : Size: torch.Size([512, 512]) : values : tensor([[-0.0283,  0.0072,  0.0138,  ...,  0.0258,  0.0164, -0.0325],
        [-0.0078,  0.0153,  0.0149,  ...,  0.0224,  0.0215, -0.0200]],
       device='mps:0'